## Import libraries

In [1]:
import logging
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
logging.getLogger('tensorflow').setLevel(logging.ERROR)

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import random as rd
#from keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,roc_curve,auc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D,BatchNormalization
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
import itertools
import warnings
warnings.filterwarnings("ignore")

## Data load and review

In [2]:
train=pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test=pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
sample_sub=pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")

In [3]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
x_train=train.drop(columns="label",axis=1)
y_train=train["label"]

## Visualization

In [5]:
fig=px.histogram(x=y_train,text_auto=True,color_discrete_sequence=['royalblue'])
fig.update_layout(bargap=.1,
                  xaxis=dict(type="category",categoryorder="total descending"),
                  title_font_color="darkblue",
                  title="Number of images",
                  xaxis_title="Image labels",
                  height=480,
                  width=700)
fig.update_traces()
fig.show()


In [6]:
img=[]
for i in range(max(y_train.value_counts().index)+1):   
    index=rd.choice(y_train[y_train.values==i].index.values)
    image=x_train.iloc[index].to_numpy().reshape((28,28))
    img.append(image)
fig=px.imshow(np.array(img),facet_col=0, binary_string=True, facet_col_wrap=5, labels={'facet_col':'Number'})
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
fig.update_layout(title="Random picture of each number",
                 height=500,
                 width=800)
fig.show()

## Preprocessing

In [7]:
X_train=x_train/255.0
Test=test/255.0
print("x train shape: ",X_train.shape)
print("test shape: ",Test.shape)

x train shape:  (42000, 784)
test shape:  (28000, 784)


In [8]:
X_train=x_train.values.reshape(-1,28,28,1)
Test=test.values.reshape(-1,28,28,1)
print("x train shape: ",X_train.shape)
print("test shape: ",Test.shape)

x train shape:  (42000, 28, 28, 1)
test shape:  (28000, 28, 28, 1)


In [9]:
Y_train=to_categorical(y_train,num_classes=10)

In [10]:
X_train,X_val,Y_train,Y_val=train_test_split(X_train,Y_train,test_size=0.2,random_state=42)
print(f'Shape of training data: {X_train.shape}')
print(f'Shape training labels: {Y_train.shape}')
print(f'Shape of validation data: {X_val.shape}')
print(f'Shape of valiation labels: {Y_val.shape}')
print(f'Shape of testing data: {test.shape}')

Shape of training data: (33600, 28, 28, 1)
Shape training labels: (33600, 10)
Shape of validation data: (8400, 28, 28, 1)
Shape of valiation labels: (8400, 10)
Shape of testing data: (28000, 784)


In [11]:
epochs=50
BATCH_SIZE = 32000 # number of sample

In [12]:
BUFFER_SIZE = 10000
BATCH_SIZE_PER_REPLICA = 1024

def get_batch_size(num_gpus=1):
    batch_size = BATCH_SIZE_PER_REPLICA * num_gpus
    print(f'BATCH_SIZE = {batch_size}')
    return batch_size

In [13]:
datagen=ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=30,
        zoom_range=0.2,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=False,
        vertical_flip=False
)        

In [14]:
train_generator = datagen.flow(
    X_train, Y_train, batch_size=BATCH_SIZE)

In [15]:
val_generator = datagen.flow(
    X_val,Y_val, batch_size=BATCH_SIZE)

## Make data compatible with GPU

In [16]:
train_data = tf.data.Dataset.from_tensor_slices((train_generator[0][0], train_generator[0][1]))
#test_data = tf.data.Dataset.from_tensor_slices((val_generator[0][0], val_generator[0][1]))

In [17]:
#train_data = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
test_data = tf.data.Dataset.from_tensor_slices((X_val, Y_val))

In [18]:
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF
train_data = train_data.with_options(options).cache().shuffle(BUFFER_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.with_options(options)

In [19]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 2


## Create model

In [20]:
#optimizer=Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999)
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

def create_model():
    model=Sequential()
    model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
    model.add(BatchNormalization())

    model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu'))
    model.add(BatchNormalization())

    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(BatchNormalization())

    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same',  activation ='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Dense(10, activation = "softmax"))
    return model
   



In [21]:
#model.compile(optimizer=optimizer,loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['sparse_categorical_accuracy'],steps_per_execution = 50)
with strategy.scope():
    model=create_model()
    model.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics=['accuracy'])
model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        1664      
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        102464    
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        3

## Train Model

In [22]:
d_train_data = train_data.batch(get_batch_size(strategy.num_replicas_in_sync))
d_test_data = test_data.batch(get_batch_size(strategy.num_replicas_in_sync))
history = model.fit(d_train_data, epochs=epochs,validation_data=d_test_data)

BATCH_SIZE = 2048
BATCH_SIZE = 2048
Epoch 1/50
16/16 [==============================] - 26s 305ms/step - loss: 1.6098 - accuracy: 0.5275 - val_loss: 5.8938 - val_accuracy: 0.4563
Epoch 2/50
16/16 [==============================] - 2s 140ms/step - loss: 0.4607 - accuracy: 0.8528 - val_loss: 2.2704 - val_accuracy: 0.6165
Epoch 3/50
16/16 [==============================] - 2s 141ms/step - loss: 0.2864 - accuracy: 0.9086 - val_loss: 1.2994 - val_accuracy: 0.7226
Epoch 4/50
16/16 [==============================] - 2s 143ms/step - loss: 0.2390 - accuracy: 0.9237 - val_loss: 0.7748 - val_accuracy: 0.8055
Epoch 5/50
16/16 [==============================] - 2s 141ms/step - loss: 0.1683 - accuracy: 0.9475 - val_loss: 0.1784 - val_accuracy: 0.9514
Epoch 6/50
16/16 [==============================] - 2s 141ms/step - loss: 0.1399 - accuracy: 0.9558 - val_loss: 0.8158 - val_accuracy: 0.8074
Epoch 7/50
16/16 [==============================] - 2s 142ms/step - loss: 0.1101 - accuracy: 0.9668 - val_loss:

## Evaluation

In [23]:
loss=pd.DataFrame({"Loss":history.history["loss"],"Val_loss":history.history['val_loss']})
fig = px.line(loss)
fig.update_layout(
                 title="Loss",
                 height=300,
                 width=500,
                 xaxis=dict(title="Epoch"),
                 yaxis=dict(title="Rate"),
                 legend_title="")
fig.show()

accuracy=pd.DataFrame({"Accuracy":history.history['accuracy'],"Val_accuracy":history.history['val_accuracy']})
fig = px.line(accuracy)
fig.update_layout(
                 title="Accuracy",
                 height=300,
                 width=500,
                 xaxis=dict(title="Epoch"),
                 yaxis=dict(title="Rate"),
                 legend_title="")
fig.show()

In [24]:
Y_pred = model.predict(X_val)
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
Y_true = np.argmax(Y_val,axis = 1) 
confusion=pd.DataFrame({"Y_true":Y_true,"Y_pred":Y_pred_classes})
fig = px.density_heatmap(confusion, x="Y_pred", y="Y_true", text_auto=True)
fig.update_layout(
                 title="Confusion matrix",
                 height=450,
                 width=450,
                 xaxis=dict(tickmode="linear",title="Y Prediction"),
                 yaxis=dict(tickmode="linear",title="Y Real"))
fig.show()

## Predict and submission

In [25]:
y_pred=model.predict(Test)

In [26]:

y_pred=np.argmax(y_pred,axis=1)
sample_sub.Label=y_pred
sample_sub.to_csv('cnn2_submission.csv',index=False)
pd.read_csv('cnn2_submission.csv')


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
